# Options of the generated tetrahedra-grid

In this notebook we look over the options for the generation of the interpolation grid. The main executable test_gorilla_main.x
takes as input for the grid generation the fortran namelist provided in the tetra_grid.inp-file. One can per hand change the values
of those namelist-variables directly and start the exectuable afterwards by hand.

However, via the the f90nml package one can control the grid-options in a python script which then allows to integrate GORILLA
in existing python routines. This is what we will demonstrate here.

First we load the f90nml package. Use either pip 

In [3]:
import f90nml